<a href="https://colab.research.google.com/github/yugokatsuki/Kaggle/blob/main/start_kaggle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()
    
drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
print(results)
print(kaggle_api_key)

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)


{'files': [{'id': '1VPZvpe5aOfNJNuKxlnOiSN0DTxD6JUIg'}]}
[{'id': '1VPZvpe5aOfNJNuKxlnOiSN0DTxD6JUIg'}]
Download 100%.


In [3]:
  !pip install kaggle


In [4]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         87           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        177           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        295           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2143           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      16866            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4593           False  
connectx

In [5]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 5.49MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 23.9MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 58.0MB/s]


In [6]:
import numpy as np
import pandas as pd
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import time

def parse(df):
    # Nan を -1 に置換
    df["Age"] = df["Age"].fillna(-1)
    df["Embarked"] = df["Embarked"].fillna(-1)
    # 文字列を数値に変換
    df["Sex"][df["Sex"] == "male"] = 0
    df["Sex"][df["Sex"] == "female"] = 1
    df["Embarked"][df["Embarked"] == "S" ] = 0
    df["Embarked"][df["Embarked"] == "C" ] = 1
    df["Embarked"][df["Embarked"] == "Q"] = 2
    return df

def split_val(x,y,rate,seed=None):
  #print(f"rate = {rate}")
  #print("rate = {}".format(rate))
  
  if seed is not None:
    np.random.seed(seed)
  N = x.shape[0]
  perm = np.random.permutation(N)
  valnum = int(rate * N)
  index1 = perm[:(-valnum)]
  index2 = perm[(-valnum):]
  return x[index1],y[index1],x[index2],y[index2]

def load_data():
    train_csv = pd.read_csv('train.csv')
    #before inputting the data, extract the value 
    train_csv = parse(train_csv)
    train_x = train_csv[['Pclass', 'Sex', 'Fare','SibSp', 'Parch', 'Age', 'Embarked']].values
    train_y = train_csv['Survived'].values
    return split_val(train_x,train_y,0.1)


In [7]:
def create_model(input_placeholder,u_dim,layer_num,y_dim,training):
    h = input_placeholder
    for i in range(layer_num):
        h = tf.layers.dense(inputs=h, units=u_dim,activation=tf.nn.relu)
        #training: flag  inference or not
        h = tf.layers.batch_normalization(h,training=training)  
    h = tf.layers.dense(inputs=h, units=y_dim)
    return h

np.random.seed(0)
tf.reset_default_graph()
#from tensorflow.keras import backend

y_dim     = 2
# hyperparameters
u_dim     = 100
layer_num = 2
epoch     = 1000
log_freq  = 100
batchsize = 200
lr        = 0.001

# data load
train_x,train_y,val_x,val_y = load_data()
N  = train_y.shape[0]
Nv = val_y.shape[0]
print('train num: {}, val num: {}'.format(N,Nv))

# calculation graph
x = tf.placeholder(tf.float32, [None, train_x.shape[1]],"input")
y = tf.placeholder(tf.int32, [None])

with tf.variable_scope("model"):
  train_z = create_model(x,u_dim,layer_num,y_dim,training=True)
with tf.variable_scope("model", reuse=True):
  z       = create_model(x,u_dim,layer_num,y_dim,training=False)

cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=train_z)

extra_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_ops):
  train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

pred_y = tf.cast(tf.argmax(z, 1), tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred_y, y), tf.float32))

# main loop
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
T = time.time() 
for ep in range(1,epoch+1) :
  perm=np.random.permutation(N)
  for i in range(0,N,batchsize):
    batch_xs=train_x[perm[i:i+batchsize]]
    batch_ys=train_y[perm[i:i+batchsize]]
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
  # monitor
  if ep%log_freq ==0 :      
    #x:placeholder
    train_loss, train_acc = sess.run([cross_entropy, accuracy], feed_dict={x: train_x, y: train_y})
    val_loss, val_acc     = sess.run([cross_entropy, accuracy], feed_dict={x: val_x, y: val_y})
    epochT = time.time()-T
    print('Epoch: %d, Time :%.4f (s), train_loss: %f,  train_acc: %f, val_loss: %f,  val_acc: %f' % (ep, epochT, train_loss, train_acc, val_loss, val_acc))
    T = time.time()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_l

train num: 802, val num: 89
Epoch: 100, Time :1.5334 (s), train_loss: 0.411716,  train_acc: 0.799252, val_loss: 0.475964,  val_acc: 0.764045
Epoch: 200, Time :1.1986 (s), train_loss: 0.425866,  train_acc: 0.806733, val_loss: 0.483509,  val_acc: 0.775281
Epoch: 300, Time :1.1805 (s), train_loss: 0.385823,  train_acc: 0.805486, val_loss: 0.425397,  val_acc: 0.831461
Epoch: 400, Time :1.1826 (s), train_loss: 0.428177,  train_acc: 0.775561, val_loss: 0.564936,  val_acc: 0.685393
Epoch: 500, Time :1.2280 (s), train_loss: 0.392856,  train_acc: 0.814214, val_loss: 0.470592,  val_acc: 0.752809
Epoch: 600, Time :1.1716 (s), train_loss: 0.390268,  train_acc: 0.815461, val_loss: 0.473846,  val_acc: 0.775281
Epoch: 700, Time :1.1313 (s), train_loss: 0.379735,  train_acc: 0.785536, val_loss: 0.495180,  val_acc: 0.707865
Epoch: 800, Time :1.1898 (s), train_loss: 0.427475,  train_acc: 0.761845, val_loss: 0.537348,  val_acc: 0.719101
Epoch: 900, Time :1.1996 (s), train_loss: 0.392455,  train_acc: 0.82

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Activation

y_dim     = 2
# hyperparameters
u_dim     = 100
layer_num = 2
epoch     = 1000
log_freq  = 100
batchsize = 200
lr        = 0.00

# data load
train_x,train_y,val_x,val_y = load_data()
print(f'train_x.shape = {train_x.shape}')
print(f'train_y.shape = {train_y.shape}')
print(f'val_y.shape = {val_x.shape}')
print(f'val_y.shape = {val_y.shape}')

N  = train_y.shape[0] #802
Nv = val_y.shape[0] #89
print(f'train num: {N}, val num: {Nv}')
model = Sequential()
model.add(Dense(u_dim, input_dim=7))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(u_dim))
model.add(BatchNormalization())
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

total_x = np.concatenate([train_x, val_x], 0)
total_y = np.concatenate([train_y, val_y], 0)

model.fit(total_x,total_y,epochs=epoch, batch_size=batchsize)

#score = model.evaluate(x_test, y_test, batch_size=128)
#print(score)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_l

train_x.shape = (802, 7)
train_y.shape = (802,)
val_y.shape = (89, 7)
val_y.shape = (89,)
train num: 802, val num: 89
Train on 891 samples
Epoch 1/1000
891/891 [==============================] - 0s 94us/sample - loss: 0.7209 - accuracy: 0.5657
Epoch 2/1000
891/891 [==============================] - 0s 21us/sample - loss: 0.6043 - accuracy: 0.6902
Epoch 3/1000
891/891 [==============================] - 0s 19us/sample - loss: 0.5945 - accuracy: 0.6981
Epoch 4/1000
891/891 [==============================] - 0s 19us/sample - loss: 0.5629 - accuracy: 0.7138
Epoch 5/1000
891/891 [==============================] - 0s 19us/sample - loss: 0.5485 - accuracy: 0.7374
Epoch 6/1000
891/891 [==============================] - 0s 20us/sample - loss: 0.5711 - accuracy: 0.7419
Epoch 7/1000
891/891 [==============================] - 0s 18us/sample - loss: 0.5224 - accuracy: 0.7340
Epoch 8/1000
891/891 [==============================] - 0s 18us/sample - loss: 0.5498 - accuracy: 0.7351
Epoch 9/1000
891/891 

In [18]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)
print(type(train_x))
test = np.concatenate([train_x, val_x], 0)
print(test.shape)




(802, 7)
(802,)
(89, 7)
(89,)
<class 'numpy.ndarray'>
(891, 7)


In [47]:
# data
test_csv = pd.read_csv('test.csv')
print(type(test_csv ))
print(test_csv.shape)
test_csv = parse(test_csv)
test_x = test_csv[['Pclass', 'Sex', 'Fare','SibSp', 'Parch', 'Age', 'Embarked']].values
prediction = model.predict(test_x)
prediction = np.round(prediction ).astype(int)
my_solution = pd.DataFrame(prediction, PassengerId, columns = ["Survived"])
# save
my_solution.to_csv("result2.csv", index_label = ["PassengerId"])



<class 'pandas.core.frame.DataFrame'>
(418, 11)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_l

In [44]:
# data
test_csv = pd.read_csv('test.csv')
test_csv = parse(test_csv)
test_x = test_csv[['Pclass', 'Sex', 'Fare','SibSp', 'Parch', 'Age', 'Embarked']].values
# predict
prediction = sess.run(pred_y, feed_dict={x: test_x})
# parse
PassengerId = np.array(test_csv["PassengerId"]).astype(int)
my_solution = pd.DataFrame(prediction, PassengerId, columns = ["Survived"])
my_solution2 = pd.DataFrame(prediction, index=PassengerId, columns = ["Survived"])
my_solution3 = pd.DataFrame(prediction, PassengerId, ["Survived"])


print(type(prediction ))
print(type( PassengerId ))
print(my_solution)
print(my_solution.index)
print(my_solution2)
print(my_solution2.index)
print(my_solution3)
print(my_solution3.index)





# save
#my_solution.to_csv("result.csv", index_label = ["PassengerId"])


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
      Survived
892          0
893          0
894          0
895          0
896          0
...        ...
1305         0
1306         1
1307         0
1308         0
1309         0

[418 rows x 1 columns]
Int64Index([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', length=418)
      Survived
892          0
893          0
894          0
895          0
896          0
...        ...
1305         0
1306         1
1307         0
1308         0
1309         0

[418 rows x 1 columns]
Int64Index([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', length=418)
      Survived
892          0
893          0
894          0
895          0
896          0
...        ...
1305         0
1306         1
1307       

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_l

# 新しいセクション

In [ ]:
!kaggle competitions submit -c titanic -f result.csv -m 'first submit'

In [48]:
!kaggle competitions submit -c titanic -f result2.csv -m 'second submit'

100% 2.79k/2.79k [00:01<00:00, 2.45kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster